# Tensorflow 2 Neural Networks

In [ ]:
# import tensorflow module. Check API version.
import tensorflow as tf

print (tf.__version__)

# required for TF to run within docker using GPU (ignore otherwise)
gpu = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpu[0], True)

## Load the data

### MNIST Dataset - Handwritten Digits (0-9)

![MNIST Sample Data](images/MnistExamples.png)

In [ ]:
# grab the MNIST dataset (may take time the first time)
print("[INFO] downloading MNIST...")

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# scale data to normalized 0.0 - 1.0 range
x_train, x_test = x_train / 255.0, x_test / 255.0

## Define Model - Keras Sequential API (Basic, Simple Models)

In [ ]:
# use tf.keras Sequential API to define simple Multi Layer Perceptron model
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),      # flatten 28 X 28 input to 784 X 1 vector
    tf.keras.layers.Dense(units=128, activation='relu'),# 128 perceptrons for first layer
    tf.keras.layers.Dense(units=10)                     # 10 units represent output classes
])

## Compile Model

In [ ]:
# compile the model
model.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.01),          # Stochastic Gradient Descent
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# print model summary
model.summary()

## Train Model

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
model.fit(x_train, y_train, epochs=20)

## Evaluate Model

In [ ]:
loss, accuracy = model.evaluate(x_test,  y_test, verbose=2)

## Predict Output

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# set up matplotlib fig, and size it to fit 3x4 pics
nrows = 1
ncols = 5
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)

# predict images from validation set
n_images = 5
test_images = x_test[:n_images]
predictions = model.predict(test_images)

# visualize image and model prediction
for i in range(n_images):
    # set up subplot; subplot indices start at 1
    sp = plt.subplot(nrows, ncols, i + 1, title="pred: %i" % np.argmax(predictions[i]))
    sp.axis('Off') # don't show axes (or gridlines)
    plt.imshow(np.reshape(test_images[i], [28, 28]), cmap='gray')

plt.show()

## Extend Model

In [ ]:
# add softmax layer to output 
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

probability_model.summary()

In [ ]:
print (model.predict(x_test[5].reshape(1, 28, 28)))

In [ ]:
print (probability_model.predict(x_test[5].reshape(1, 28, 28)))

In [ ]:
print (np.argmax(probability_model.predict(x_test[5].reshape(1, 28, 28))))

## Define Model - Keras Functional API (Advanced, Complex Models)

In [ ]:
# use tf.keras Functional API to define simple Multi Layer Perceptron model
x_i = tf.keras.layers.Input(shape=(28, 28))                 # input 28 X 28 single channel image
x1  = tf.keras.layers.Flatten()(x_i)                        # flatten 28 X 28 input to 784 X 1
x2  = tf.keras.layers.Dense(units=128, activation='relu')(x1) # 128 perceptrons for first layer
x3  = tf.keras.layers.Dense(units=10)(x2)                   # 10 units represent output classes
x_o = tf.keras.layers.Softmax()(x3)                         # softmax function for class probabilities

model2 = tf.keras.Model(inputs=x_i, outputs=x_o)

In [ ]:
# compile the model
model2.compile(
    optimizer=tf.keras.optimizers.SGD(lr=0.01),          # Stochastic Gradient Descent
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

# print model summary
model2.summary()

In [ ]:
# run training
model2.fit(x_train, y_train, epochs=20)

In [ ]:
# show the accuracy on the testing set
loss, accuracy = model2.evaluate(x_test,  y_test, verbose=2)